Pasos para optimizacion de procesos

- Acceso a datos y manipulacion de los datos 
  - Lectura de archivo original 
  - Busqueda de datos segun criterios en filas y columnas 
  - Filtro con las columnas necesitadas 
- Creacion de rango de consulta en columnas 
- Lectura de numeros mediante el uso de OCR y reconocimiento de imagenes
  - Mediante el uso de librerias de reconocimiento de imagenes se extrae el texto 
  - Se crea una seleccion especifica en el area especificada de la imagen
    - se manipula la imagen seleccionando el area especifica de texto o numero 
  - se extrae el texto o se digita manualmente  
- Consulta a base de datos con los datos obtenidos 
  - Se establece una conexxion a base de datos con los parametros establecidos
  - Utillizando la informacion obtenida se realiza la consulta y se obtienen los datos devueltos 
- Creacion de DataFrame de acuerdo a las condiciones establecidas
  - se realiza un modelo que reune:
    - la informacion original, ubicada en el orden respectivo de columnas
    - se establece la informacion en las coluymnas requeridas de acuerdo a los datos obtenidos en la base de datos 
    - se pone la debida informacion de acuerdo a las condiciones dadas.

In [11]:
#! pip remove mysql-connector-python

#! pip freeze > requirements.txt
# ! pip install -r requirements.txt
#! pip install flask_mysqldb

In [10]:
import numpy as np
import pandas as pd
import requests
import os

csvFile = pd.read_csv('mt1.csv')

csvFile[['FotoStickerFemsa','CodifoclienteKof_Mex','Tiene match?']].head()
csvFile[['FotoStickerFemsa'][0]]

0       https://shortextraction.grupomeiko.io/20230630...
1       https://shortextraction.grupomeiko.io/20230630...
2       https://shortextraction.grupomeiko.io/20230630...
3       https://shortextraction.grupomeiko.io/20230630...
4       https://shortextraction.grupomeiko.io/20230630...
                              ...                        
6178    https://shortextraction.grupomeiko.io/20230403...
6179    https://shortextraction.grupomeiko.io/20230403...
6180    https://shortextraction.grupomeiko.io/20230403...
6181    https://shortextraction.grupomeiko.io/20230403...
6182    https://shortextraction.grupomeiko.io/20230404...
Name: FotoStickerFemsa, Length: 6183, dtype: object

In [36]:
f1=csvFile.tail(5)

f1=f1.to_numpy()

f1.shape


numpy.ndarray

In [5]:

#csvFile.info()# show info type data frame

#type(csvFile.columns)

data=csvFile.head()

print(data)

                           Place_id   Ciclo  idRegistroEncuesta  \
0  e6a76b04bf2491c7b29d07abd4ff7af8     623             6865944   
1  1398082a5e7548eeeecd1f2353aadce8     623             6865940   
2  3ff1065da3fdd1b2cd827e2727b2f3ef     623             6865833   
3  ba2559ced84fb382e31cd189d03c495c     623             6865832   
4  6db38c59aa36597f98125afc76cbf822  subido             6865829   

   idEstablecimiento    NombreEstablecimiento               Direccion  \
0             980606   mivelanea la escondida        ZUMPANGO.46.1.1.   
1             980602  Abarrotes Rosales Rojas        ZUMPANGO.46.1.1.   
2             980598    antobitog msry y meme  NEZAHUALCOYOTL.1.4.3.B   
3             980597           alitad y beear  NEZAHUALCOYOTL.1.4.3.A   
4             928076              el progreso    CHIMALHUACAN.1.3.6.A   

                                    FotoStickerFemsa  Imagen  \
0  https://shortextraction.grupomeiko.io/20230630...     NaN   
1  https://shortextraction.gru

In [4]:
#filter just empy fields in 'CodifoclienteKof_Mex'
data1=csvFile.loc[csvFile['CodifoclienteKof_Mex'].notna()==False,['idEstablecimiento','FotoStickerFemsa','CodifoclienteKof_Mex']].head(5) #obtenenemos las filas que no tienen Codigo Aun

#print the selected colums and description
data1.info()
print(data1.values[0][1])
print(data1.values[0][1]=='https://shortextraction.grupomeiko.io/20230626/626_3596995_FotoStickerFemsa.jpg')
#csvFile[['FotoStickerFemsa']].head(5) #obtenenemos las filas que no tienen Codigo Aun

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 978 to 983
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   idEstablecimiento     5 non-null      int64 
 1   FotoStickerFemsa      5 non-null      object
 2   CodifoclienteKof_Mex  0 non-null      object
dtypes: int64(1), object(2)
memory usage: 160.0+ bytes
https://shortextraction.grupomeiko.io/20230626/626_3596995_FotoStickerFemsa.jpg
True


In [5]:
#request to extract a specifi image
from PIL import Image
from io import BytesIO


req = requests.get(data1.values[0][1])
if req.status_code==200 :
    code=data1.values[0][0]
    nameImage=f'Img_{code}.jpg'
    path=f'tmp/{nameImage}'
    with open (path,'wb') as file:
        file.write(req.content)
    print('Download complete!')
    #i = Image.open(BytesIO(req.content))
else: print(f'Error Status Code {req.status_code}')

Download complete!


In [9]:
from mysql.connector import connect , Error

class DB():
    def __init__(self):
        self.host='meiko-prod.cmebrzxfmsvx.us-east-2.rds.amazonaws.com'
        self.user='support_data'
        self.password='puHnBWtMOzQzHarQMhnmhooy'
        self.port=3306
        
        
    def _newQuery(self, query):
        try:
            with connect(
                host= self.host,
                user= self.user,
                password=self.password,
                #self.port
            ) as connection:
                with connection.cursor() as cursor:
                    return cursor.execute(query)
        except Error as e:
            return f'An Error ocurred {e}'
        
        
import mysql.connector

config = {
  'user': 'support_data',
  'password': 'puHnBWtMOzQzHarQMhnmhooy',
  'host': 'meiko-prod.cmebrzxfmsvx.us-east-2.rds.amazonaws.com',
  'database': 'jobOrder',
  'port':3306,
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)
query = ("SELECT * FROM jobOrder.sp_mega_job_nf WHERE anio =2023 LIMIT 10")
cursor=cnx.cursor()
cursor.execute(query)

print(type(cursor))
for row in cursor:
    print(row)

cursor.close()
cnx.close()
        

#newq=DB()  
#print(newq._newQuery('SELECT * FROM jobOrder.sp_mega_job_nf'))

<class 'mysql.connector.cursor_cext.CMySQLCursor'>
<class 'list'>
[('265', '#I|265#N|170#B|1#C|10', datetime.datetime(2022, 12, 21, 17, 4, 48), 'Inactivo', 'COLOMBIA', '1', 2023, 'EDGE', 'BEBIDAS NO ALCOHOLICAS', 'CATEGORIA', 'LECHE LIQUIDA', '#I|265#N|170#B|1#C|10', 'TASA', 'BARRANQUILLA,BOGOTA,BUCARAMANGA,CALI,CARTAGENA,EJE CAFETERO,IBAGUE,MEDELLIN,NEIVA,SANTA MARTA', 'Autoservicio,Cafeteria,Panaderia,Tienda', 'ALPINA,ALQUERIA', None, '2022/12/21 08:31:31', 'EDGE', None, None), ('265', '#I|265#N|170#B|1#C|10#M|0#R|0', datetime.datetime(2022, 12, 21, 17, 4, 48), 'Inactivo', 'COLOMBIA', '1', 2023, 'EDGE', 'BEBIDAS NO ALCOHOLICAS', 'CATEGORIA,FABRICANTE,MARCA', 'LECHE LIQUIDA,OTROS FABRICANTES,OTRAS MARCAS', '#I|265#N|170#B|1#C|10#M|0#R|0', 'CARAS REGULAR,PRESENCIA,PRESENCIA DECLARADA', 'CALI,EJE CAFETERO', 'Autoservicio,Cafeteria,Panaderia,Tienda', 'ALPINA,ALQUERIA', None, '2022/12/21 08:31:31', 'EDGE', None, None), ('265', '#I|265#N|170#B|1#C|10#M|0#R|0#A|8_3', datetime.datetime(2022,

In [ ]:
################ MApeo de datos Hash verification
import mysql.connector
import pprint

config = {
  'user': 'support_data',
  'password': 'puHnBWtMOzQzHarQMhnmhooy',
  'host': 'meiko-prod.cmebrzxfmsvx.us-east-2.rds.amazonaws.com',
  'database': 'jobOrder',
  'port':3306,
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

querySpMegaJob = (f"""SELECT
	DISTINCT llave_job AS HASHMGJ, ll.tipos_medicion 
FROM
	jobOrder.sp_mega_job_nf ll
WHERE
	estado_job = '{status}'
	AND ciclo = '{cycle}'
	AND anio = '{anio}'
	AND pais = '{country}'
	AND estudio  = '{study}'
	AND tipos_medicion LIKE '%%'
	AND empresas LIKE '%{company}%';""")
cursor=cnx.cursor()
cursor.execute(querySpMegaJob)
result=cursor.fetchall()
print(type(result))
newdict={}
for row in result:
    typesMed=str(row[1]).split(',')
    for meditionType in typesMed:
        #print(meditionType)
        if not meditionType in newdict.keys():
            newdict[meditionType]=[]
        
        if meditionType in row[1]:
          newdict[meditionType]=newdict[meditionType]+[row[0]]
  
queryViewMapeoDatos="""

"""  
#c=1          
#for i in newdict:
#  print(i[c],'\n')
#  c+=1            

#print(newdict)

pprint.pprint(newdict)

# Sets 'pretty_dict_str' to the formatted string value
#pretty_dict_str = pprint.pformat(newdict)

cursor.close()
cnx.close()


#newq=DB()  
#print(newq._newQuery('SELECT * FROM jobOrder.sp_mega_job_nf'))
